In [7]:
import pandas as pd
from sqlalchemy import create_engine
#from config import rds_connection_string

## Extract

In [8]:
rawdata_2008 = pd.read_csv('Data/data.csv')
rawdata_2018 =pd.read_csv('Data/zipcode_sales_nov2018.csv')

In [9]:
rawdata_2008

,street,city,zip,state,beds,baths,sq_ft,type,price,latitude,longitude,day,month,date,year
0,1 KENNELFORD CIR,SACRAMENTO,95823,CA,3,2,1144,Residential,200345,38.464520,-121.427606,Mon,May,19,2008
1,10 SEA FOAM CT,SACRAMENTO,95831,CA,3,3,2052,Residential,415000,38.487885,-121.545947,Wed,May,21,2008
2,100 CHELSEA CT,FOLSOM,95630,CA,3,2,1905,Residential,500000,38.694350,-121.177259,Mon,May,19,2008
3,100 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,4897,38.886091,-121.289744,Mon,May,19,2008
4,100 CRYSTALWOOD WAY,LINCOLN,95648,CA,0,0,0,Residential,4897,38.886282,-121.289706,Mon,May,19,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,993 MANTON CT,GALT,95632,CA,4,3,2307,Residential,300000,38.272942,-121.289148,Tue,May,20,2008
979,9937 BURLINE ST,SACRAMENTO,95827,CA,3,2,1092,Residential,150000,38.559641,-121.323160,Fri,May,16,2008
980,9949 NESTLING CIR,ELK GROVE,95757,CA,3,2,1543,Residential,275000,38.397455,-121.468391,Fri,May,16,2008
981,9967 HATHERTON WAY,ELK GROVE,95757,CA,0,0,0,Residential,222500,38.305200,-121.403300,Fri,May,16,2008


## Transform

In [10]:
def clean_2018_data(rawdata_2018):
    #rename column headers to improve presentation and prepare for merge
    cleandata_2018 = (rawdata_2018.copy()
                         .rename(columns={"ZIP":'zip',
                                          "Sales":'Sales_2018',
                                          '2017 median':'Median_Price_2017',
                                          '2018 median':'Median_Price_2018'})
                         .drop(['Change'],axis=1))
    return cleandata_2018

In [11]:
def clean_2008_data(rawdata_2008):
    cleandata_2008 = rawdata_2008.copy()
    #drop any rows listing house size as 0 sqf
    cleandata_2008 = cleandata_2008.drop(cleandata_2008.index[cleandata_2008['sq_ft']==0])
    
    # determine the price per sqft
    cleandata_2008['price per sqft'] = cleandata_2008['price']/cleandata_2008['sq_ft']
    
    return cleandata_2008

In [12]:
def aggregate_2008_data(cleandata_2008):
    aggregate_2008 = (round(cleandata_2008.copy()
         #group sales by zip code
        .groupby('zip')
         # find the medians of each category, and the total sales count                   
        .agg({'price':'median',
              'price per sqft':'median',
              'sq_ft':'median',
              'city':'count'})
         # rename column headers to reflect the new meanings                   
        .rename(columns={'city':"Sales_2008",
                         'price':"Median_Price_2008",
                         'price per sqft':'Median_Price_sqft_2008',
                         'sq_ft':"Median_sqft_2008"}),0)
                     )
    return aggregate_2008

## Load

In [17]:
def export_data(data_2008,data_2018):
    #export final dataframe into a csv file
    combined_stats = pd.merge(data_2008, data_2018, on="zip",how='inner')
    combined_stats.to_csv('Data/zipcode_medians_combined.csv')
    
    return combined_stats

In [18]:
def load_data(data):
    #load data into pgAdmin 4 using sqlalchemy and pandas
    engine = create_engine(f'postgresql://{rds_connection_string}')
    combined_df.to_sql(name='zippy', con=engine, if_exists='append', index=False)

## Code Execution

In [19]:
clean2018 = clean_2018_data(rawdata_2018)
clean2008 = clean_2008_data(rawdata_2008)
aggregate2008 = aggregate_2008_data(clean2008)
combined_df = export_data(aggregate2008,clean2018)
load_data(combined_df)

combined_df.head()

,zip,Median_Price_2008,Median_Price_sqft_2008,Median_sqft_2008,Sales_2008,Community,Sales_2018,Median_Price_2017,Median_Price_2018
0,95603,285000.0,207.0,1255.0,3,Auburn,29,407550,440000
1,95608,248567.0,182.0,1472.0,20,Carmichael,61,372250,399500
2,95610,219000.0,123.0,1410.0,7,Citrus Heights,29,325000,362000
3,95621,168646.0,143.0,1268.0,28,Citrus Heights,32,306000,316500
4,95624,229000.0,139.0,1721.0,31,Elk Grove,74,390000,424000
